# COGS 108 - Data Checkpoint

# Names

- Stephen Kim
- Clara Yi
- Ethan Lee
- Ernest Lin
- Wesley Nguyen

<a id='research_question'></a>
# Research Question

Do the macroscopic socioeconomic features of a state, specifically median income, percentage of population without health insurance, and labor breakdown, have a correlation to COVID mortality rate in 2020-2021?

# Dataset(s)



### Dataset 1

- Dataset Name: United States COVID-19 Cases and Deaths by State over Time
- Link to the dataset: https://data.cdc.gov/Case-Surveillance/United-States-COVID-19-Cases-and-Deaths-by-State-o/9mfq-cb36/data
- Number of observations: 44,280 rows, 15 columns, 664,200 observations total

This dataset contains the United States (and underlying US territories) data for its COVID rates over time. Such rates include total cases, new cases, total deaths, new deaths, and other metrics that give an overall view of the statistics of COVID for each state. There are submission dates for each row, so that is how we are going to link the rates to specific periods of time

### Dataset 2

- Dataset Name: Employees on nonfarm payrolls by state and selected industry sector, seasonally adjusted
- Link to the dataset: https://www.bls.gov/news.release/laus.t03.htm
- Number of observations: 50 rows, 9 columns, 450 observations total

Dataset from the US Bureau of Labor Statistics, counting the total number of employees in thousands in the labor force in each state as well as in each of eight industries (construction, manufacturing, trade/transportation/utilities, finance, services, education/health, leisure/hospitality, government).

### Dataset 3

- Dataset Name: Median Household Income and Percentage of Americans without Health Insurance in 2020
- Link to the dataset: https://docs.google.com/spreadsheets/d/174jFoW8KsXGJmpNUx8cbh6j4l6rhQhpOUKIPnkzk3lM/edit#gid=0
- Number of observations: 50 rows, 2 columns, 100 observations total

This dataset contains the United States' for the median household income and percentage of Americans without Health Insurance in 2020. This data was taken from two different sources, [United States Census Bureau Website](https://www.census.gov/quickfacts/fact/map/CA/HEA775220) and [Federal Reserve Economic Data](https://fred.stlouisfed.org/release/tables?rid=249&eid=259515&od=2020-01-01#), and all of this data was manually imported into a Google SHeet that was converted to a CSV file. 

### Merging Data
Since we are using 3 different primary datasets, we will identify each state with a unique code (California would be CA, Missouri would be MO, etc.). Ultimately, we will merge the datasets during our analysis, with several rows of data for each state.

# Setup

In [1]:
!pip3 install pandas
!pip install pandas

You should consider upgrading via the 'c:\users\ernest\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\ernest\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd 

# Data Cleaning (Process)

Describe your data cleaning steps here.
## Dataset 1
- With the imported data, we removed unncessary states. We only want the 50 states not including territories or DC
- We then removed the columns that we didn't need for analysis. We did this by selecting the columns that we needed
- We also wanted the dates to appear in a sortable/searchable way, so we made the dates arranged in yyyy-mm-dd format

**Note**: Since the data is arranged by date, we created a function ```read_covid_data``` that will return the 50 states with their respective data for just that specified date
## Dataset 2
- The raw data file for Dataset 2 is an excel file. The format of the data was not organized in a way that complements dataframes, so we had a lot of unnecessary texts in the excel
read as data entries as well. 
- Our first step was to identify the columns and rows that we want, which are the 50 states. We removed unnecessary states that does not include territories and removed all the extra
non-state entries that were read as rows.
- Another problem is that some names in the State column had some numbers that were unintentionally read from the excel sheet. We solved this by removing all occurences of numbers and parentheses from the State column.
- We had to reorganize the structure of the dataframe, as the original file had the data stacked on top of each other so each state had 3 rows in the Dataframe. We did so by separating the 
raw dataframe into three different dataframe, then combining them into a single dataframe so we only have 50 rows.
- We then removed unnessary columns, such as data frome other time periods (Our focus was December of 2020). We also combined the columns of job sectors into two groups relevant to our analysis: White collar and Blue collar jobs.
- Since other datasets use state codes and the original data uses state names, we had to transform state names in the States column to their corresponding state codes. We did this by defining a function ```to_state_code``` that uses a dictionary to map each state name to their state code.
- Our final step for Dataset 3 is to export the cleaned dataset as a csv and save it to the "Cleaned Data" folder.
## Dataset 3
For Dataset 3, we have two primary steps in cleaning the data. The first step, which was manually inputting the data from the data sources to a CSV file via Google Sheets. This manual step was necessary due to the fact that the original data source did not have an option to directly extract/download the raw data. Since there were only 50 observations, we decided manual input was the best option. 

Our second step for Dataset 3 was to import the data into this notebook. We uploaded the CSV file into our "Raw Data" folder, and then used read_csv to bring it into a dataframe, which is a usable format for our future analysis. After making sure there were no issues, we then saved it to the "Cleaned Data" folder.

In [3]:
# Cleaning State Data
def clean_covid_data():
    # Date Closure
    def apply_date(date: str) -> str:
        split_date = date.split("/")
        return "/".join([split_date[2], split_date[0], split_date[1]])
        
    
    # Read the data (already in tabular form)
    covid_data_url = r".\Data\State Data\United_States_COVID-19_Cases_and_Deaths_by_State_over_Time.csv"
    covid_data = pd.read_csv(covid_data_url)
    
    # States we will not be looking at (These aren't part of the 50 states)
    remove_states = ["RMI", "FSM", "GU", "MP", "PW", "NYC", "PR", "AS", "VI", "DC"]
    covid_data = covid_data[~covid_data["state"].isin(remove_states)]
    
    # Remove columns we don't need
    covid_data = covid_data[["submission_date", "state", "tot_cases", "tot_death"]]
    
    # Change Date format to allow for easier sorting
    covid_data["submission_date"] = covid_data["submission_date"].apply(apply_date)
    
    # Sort Date
    covid_data.sort_values("submission_date", inplace=True, ascending=False)
    covid_data.reset_index(inplace=True, drop=True)
    
    # Save Data
    clean_covid_data_url = r".\Cleaned Data\state_covid_data.csv"
    covid_data.to_csv(clean_covid_data_url, index=False)
    
clean_covid_data()

FileNotFoundError: [Errno 2] File b'.\\Data\\State Data\\United_States_COVID-19_Cases_and_Deaths_by_State_over_Time.csv' does not exist: b'.\\Data\\State Data\\United_States_COVID-19_Cases_and_Deaths_by_State_over_Time.csv'

In [ ]:
def read_covid_data(month: int, day: int, year: int):
    covid_data_url = r".\Cleaned Data\state_covid_data.csv"
    covid_data = pd.read_csv(covid_data_url)
    
    date_filter = formatDate(month, day, year)
    covid_data = covid_data[covid_data["submission_date"] == date_filter]
    covid_data.sort_values("state", inplace=True)
    covid_data.reset_index(inplace=True, drop=True)
    
    return covid_data

def formatPreZero(num: int) -> str:
    if num >= 10:
        return str(num)
    
    return "0" + str(num)
    
    
def formatDate(month: int,  day: int, year: int) -> str:
    return f"{year}/{formatPreZero(month)}/{formatPreZero(day)}"

read_covid_data(3, 15, 2021)

,submission_date,state,tot_cases,tot_death
0,2021/03/15,AK,58212,331
1,2021/03/15,AL,507479,10798
2,2021/03/15,AR,327060,5481
3,2021/03/15,AZ,834006,16553
4,2021/03/15,CA,3528795,55330
5,2021/03/15,CO,452758,6040
6,2021/03/15,CT,293102,7788
7,2021/03/15,DE,91768,1511
8,2021/03/15,FL,1943062,33574
9,2021/03/15,GA,1034763,18262


In [ ]:
# Cleaning socioeconomic data

socioeconomic_data_url = r'./Raw Data/socioeconomic_data.csv'
socioeconomic_data = pd.read_csv(socioeconomic_data_url)
print(socioeconomic_data)

# Saving to GitHub
clean_socioeconomic_data_url = r"./Cleaned Data/clean_socioeconomic_data.csv"
socioeconomic_data.to_csv(clean_socioeconomic_data_url, index=False)

   State  Persons without Health Insurance, % Median Household Income in 2020
0     AL                                 11.7                          54,393
1     AK                                 13.9                          74,476
2     AZ                                 13.6                          66,628
3     AR                                 10.9                          50,540
4     CA                                  8.9                          77,358
5     CO                                  9.3                          82,611
6     CT                                  7.0                          79,043
7     DE                                  8.1                          69,132
8     FL                                 16.3                          57,435
9     GA                                 15.5                          58,952
10    HI                                  5.0                          80,729
11    ID                                 12.8                   

In [3]:
#Copied from https://gist.github.com/rogerallen/1583593
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

def to_state_code(state_name):
    return us_state_to_abbrev[state_name]

In [4]:
def clean_labor_data():
    #Read excel file, renames first column to States and take out null rows
    raw_labor_data = pd.read_excel("./Raw Data/collar_dataset_raw.xlsx", header = 4)
    raw_labor_data.rename(columns={"Unnamed: 0": "State"}, inplace=True)
    raw_labor_data = raw_labor_data.dropna()

    #Take out data from 2021 and only keep 2020
    raw_labor_data = raw_labor_data[["State", "Dec.\n2020", "Dec.\n2020.1", "Dec.\n2020.2"]]
    
    non_states = ["Virgin Islands", "District of Columbia", "Puerto Rico"]

    #Removes all non official states from dataset
    for region in non_states:
        raw_labor_data = raw_labor_data[raw_labor_data["State"].str.contains(region)==False]

    #Reset index to start at 0
    raw_labor_data = raw_labor_data.reset_index(drop = True)

    #eliminated extra characters in state names
    raw_labor_data["State"] = raw_labor_data["State"].str.replace('\d+', '')
    raw_labor_data["State"] = raw_labor_data["State"].str.replace('(', '')
    raw_labor_data["State"] = raw_labor_data["State"].str.replace(')', '')

    #Convert state names into codes (First two letters of each state name)
    raw_labor_data["State"] = raw_labor_data["State"].apply(lambda state_name: us_state_to_abbrev[state_name])

    #Original raw data has different columns stacked on top of each row, so we need to reorder the dataset.
    #Block 1 contains total, constructing and mining data
    block1 = raw_labor_data[:50]
    block1.columns = ["State", "Total", "Constructing", "Mining"]

    #Block 2 contains Trade, Financial and Professional
    block2 = raw_labor_data[50:100]
    block2.columns = ["State", "Trade", "Financial", "Professional"]

    #Block 3 contains Education, Leisure and Government
    block3 = raw_labor_data[100:]
    block3.columns = ["State", "Education", "Leisure", "Gov"]

    #merge all blocks into one dataframe
    labor_data = block1.merge(block2, on="State")
    labor_data = labor_data.merge(block3, on="State")

    #We only need data on white collar and blue collar, so we can combine each job sector to their respective group.
    labor_data["White_col"] = labor_data["Constructing"] + labor_data["Mining"] + labor_data["Trade"]
    labor_data["Blue_col"] = labor_data["Trade"] + labor_data["Financial"] + labor_data["Professional"] + labor_data["Education"] + labor_data["Leisure"] + labor_data["Gov"]

    #Get rid of all other columns except State, White_col, Blue_col and Total
    labor_data.drop(columns = ["Constructing", "Mining", "Trade", "Financial", "Professional", "Education", "Leisure", "Gov"], inplace=True)
    #export as csv
    labor_data.to_csv('./Cleaned Data/state_labor_data.csv')
    print(labor_data)

clean_labor_data()

   State    Total White_col Blue_col
0     AL   2012.5     736.1   1535.9
1     AK    304.1      89.7    248.3
2     AZ   2860.3     914.6   2366.0
3     AR   1257.5     464.8    964.5
4     CA  15937.9    5069.6  12827.6
5     CO   2626.7     802.0   2108.3
6     CT   1570.8     497.1   1273.9
7     DE    442.3     129.6    375.2
8     FL     8492    2683.5   7096.2
9     GA   4449.8    1536.6   3590.0
10    HI    541.7     148.1    465.7
11    ID    776.4     281.0    613.0
12    IL   5637.3    1946.0   4542.5
13    IN   3024.8    1274.9   2213.7
14    IA   1513.4     604.3   1141.2
15    KS   1354.7     485.0   1062.4
16    KY   1841.8     725.6   1432.0
17    LA   1834.6     612.2   1474.7
18    ME    601.2     196.8    488.3
19    MD   2603.9     732.4   2198.1
20    MA   3356.3     941.6   2763.6
21    MI   4027.9    1517.1   3081.0
22    MN   2719.6     931.9   2147.1
23    MS   1125.9     419.9    883.7
24    MO   2799.2     937.6   2245.9
25    MT    476.4     145.7    394.4
2

C:\Users\Ernest\AppData\Local\Temp\ipykernel_20120\1232530879.py:20: FutureWarning: The default value of regex will change from True to False in a future version.
  raw_labor_data["State"] = raw_labor_data["State"].str.replace('\d+', '')
C:\Users\Ernest\AppData\Local\Temp\ipykernel_20120\1232530879.py:21: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  raw_labor_data["State"] = raw_labor_data["State"].str.replace('(', '')
C:\Users\Ernest\AppData\Local\Temp\ipykernel_20120\1232530879.py:22: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  raw_labor_data["State"] = raw_labor_data["State"].str.replace(')', '')
